# Host-Galaxy Association with Pröst

### This notebook shows the basics of prost for host-galaxy association.

First, let's import some relevant packages. We'll need distributions to define our priors and likelihoods.

In [1]:
import pandas as pd
from scipy.stats import gamma, halfnorm, uniform
from astropy.cosmology import LambdaCDM
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 

#pretty plotting
sns.set_context("poster")

#enable interactive plotting
%matplotlib inline

Pröst also provides a custom distribution object for the expected redshift of a transient with a given brightness and fixed volumetric rate. 

In [2]:
from astro_prost.helpers import PriorzObservedTransients, SnRateAbsmag

Next, we import the functions that do the bulk of the work.

In [3]:
from astro_prost.associate import associate_sample, prepare_catalog

Now let's read in a transient catalog, which should have at least the name and coordinates of the transients. Here, we use the [ZTF BTS](https://sites.astro.caltech.edu/ztf/bts/explorer.php?f=s&subsample=trans&classstring=&endpeakmag=19.0&purity=y&quality=y) sample.

In [4]:
transient_catalog = pd.read_csv("../../src/astro_prost/data/ZTFBTS_TransientTable.csv")

#only take the first 10 events 
transient_catalog = transient_catalog.sample(n=10)

print(transient_catalog.sample(frac=1)[['IAUID', 'RA', 'Dec']])

          IAUID           RA          Dec
4736  SN2023gaj  19:50:44.47  +57:33:14.6
836   SN2019hte  14:34:13.60  +21:36:05.9
3165  SN2021nli  14:02:12.66  -18:45:16.1
2214  SN2020thn  00:06:56.11  +45:05:29.8
2176  SN2020svi  00:30:31.84  +33:12:27.7
675   SN2019dgw  19:11:11.79  +48:29:33.6
2240  SN2020uao  01:08:47.61  +27:02:42.0
2824  SN2021dps  08:00:54.09  -02:13:38.5
3763  SN2022gpw  10:47:42.19  +54:24:41.5
4167  SN2022rjx  00:19:10.68  +29:19:56.3


Next, we define the priors for the association. By default, Pröst defines priors on a transient's observed(!)
* Redshift distribution
* Fractional radial offset from its host galaxy (defined in units of the host's [Directional Light Radius](https://arxiv.org/pdf/1604.06138))
* Host galaxy brightness, in absolute magnitude ($B$-band if associating with the glade catalog, else the median across $griz$)

We'll keep things simple for now, and assume that we detect fewer events with redshift, with broad uniform priors for brightness and fractional offset. 

In [5]:
# define priors for properties
priorfunc_z = halfnorm(loc=0.0001, scale=0.5)
priorfunc_offset = uniform(loc=0, scale=10)
priorfunc_absmag = uniform(loc=-30, scale=20)

If, instead, you want the redshift prior to be based on an observed distribution of transients within a given absolute magnitude range, 
we can build an empirical distribution by uniformly distributed transients in a cosmological volume between $z_{min}$ and $z_{max}$, and call 
the the subset with peak brightness above $mag_{cutoff}$ to be "observed". 

By default, the code draws a transient's peak brightness from a truncated gaussian from $absmag_{min}$ to $absmag_{max}$, with mean of $absmag_{mean}$. 

In [6]:
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)
priorfunc_z = PriorzObservedTransients(z_min=0, z_max=1, mag_cutoff=19, absmag_mean=-19, absmag_min=-24, absmag_max=-17, cosmo=cosmo)

We can then plot the resulting distribution:

In [7]:
#broken for now
priorfunc_z.plot()

/Users/alexgagliano/miniconda3/envs/prost/lib/python3.12/site-packages/astro_prost/helpers.py:988: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


The pdf of the distribution can be evaluated and sampled:

In [8]:
z_samples = priorfunc_z.rvs(10)
print(z_samples)

[0.27505459 0.67008328 0.10710514 0.32168342 0.20314816 0.38112814
 0.14421972 0.02969649 0.13405895 0.08333242]


In [9]:
priorfunc_z.pdf(z_samples)

array([2.07589643, 0.02620794, 3.68362149, 1.43950389, 3.15583918,
       0.8460035 , 3.84571512, 1.00366773, 3.84074048, 3.21932454])

Next, we set the likelihoods. Note that we only set these for fractional offset and brightness; the redshift likelihood comes from comparing the photometric redshifts of candidate galaxies with the redshift of the transient (if available). 

In [10]:
likefunc_offset = gamma(a=0.75)
likefunc_absmag = SnRateAbsmag(a=-25, b=20)

priors = {"offset": priorfunc_offset, "absmag": priorfunc_absmag, "z": priorfunc_z}
likes = {"offset": likefunc_offset, "absmag": likefunc_absmag}

In [11]:
plt.hist(likefunc_offset.rvs(size=10000));
plt.xlabel(r"Fractional Offset ($\theta$/DLR)");
plt.ylabel("Likelihood PDF");

Our likelihood for fractional offset sharply peaks near 0: if a transient is sitting on top of a galaxy, odds are very good that it's the host. The likelihood for host galaxy brightness is set here to a supernova-based likelihood, which increases with absolute magnitude.

In [12]:
plt.hist(likefunc_absmag.rvs(size=1000))
plt.xlabel(r"Host Absolute Magnitude");
plt.ylabel("Likelihood PDF");

Next, let's set up the properties of the run:

In [15]:
# list of catalogs to search -- options are (in order) glade, decals, panstarrs
# If multiple are listed, the code stops whenever it finds a high-probability host
catalogs = ["panstarrs"]

# The name of the coordinate columns in the dataframe
# Can be in string hourangle, deg or decimal degrees
transient_coord_cols = ("RA", "Dec")

# the column corresponding to transient names
transient_name_col = "IAUID"

# can be 0, 1, or 2
verbose = 1

# If true, enables multiprocessing with mpire (cannot be run in this notebook)
parallel = False

# If true, saves the results of the run to disk (alternative is to return them directly)
save = False

# If true, shows a progress bar for each association (only available when parallel=True)
progress_bar = False

# If true, concatenates the source properties from the matched catalog to the returned results
cat_cols = True

In [16]:
transient_catalog = prepare_catalog(
    transient_catalog, transient_name_col=transient_name_col, transient_coord_cols=transient_coord_cols
)

# cosmology can be specified, else flat lambdaCDM is assumed with H0=70, Om0=0.3, Ode0=0.7
transient_catalog_with_hosts = \
    associate_sample(
        transient_catalog,
        priors=priors,
        likes=likes,
        catalogs=catalogs,
        parallel=parallel,
        verbose=verbose,
        save=save,
        progress_bar=progress_bar,
        cat_cols=cat_cols,
)

Associating for SN2021nli at RA, DEC = {transient.position.ra.deg:.6f}, {transient.position.dec.deg:.6f}
Removing panstarrs shreds...
Removing 21 indices from tentative matches in panstarrs!
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Association successful!
Found a good host in panstarrs!
Chosen galaxy has catalog ID of 85492105476205387and RA, DEC = {best_ra:.6f}, {best_dec:.6f}
Getting img with size len 35.18...


IndexError: list index out of range

Let's look at the results: 

In [ ]:
transient_catalog_with_hosts[['IAUID', 'host_id', 'host_ra', 'host_dec', 'host_prob', 'smallcone_prob', 'missedcat_prob']]